<a href="https://colab.research.google.com/github/staller84/minhee/blob/master/baseline(sentiment_analysis_friends).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preparation**

- Edit > Notebook settings > Hardward accelerators > GPU > SAVE
- Download the Friends dataset in EmotionLines website:
http://doraemon.iis.sinica.edu.tw/emotionlines/download.html
- Download the unlabeled json file.

# **Tutorials**

##### **Settings**

In [0]:
!pip install transformers --quiet # package installer for python tranformar packe 2가지가 반복적인 구조를 가진다.

MessageError: ignored

In [0]:
import torch
from transformers import BertModel, BertTokenizer

In [0]:
pretrained_weights = 'bert-base-uncased'  # pretrained 된걸 release해야 burt가 모두 구조적으로 동일하지만, 어떤 데이터를 얼마나 하는지 pretrain이 중요하다
# 이미 학습된 결과를 가져다 써야, 좋은 모델을 쓸 수 있다. 학습 체크 포인트. 
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
model = BertModel.from_pretrained(pretrained_weights)

##### **Tokenization**

In [0]:
sentence = 'All the classes are provided.'

In [0]:
tokens = tokenizer.tokenize(sentence)
tokens = ['[CLS]'] + tokens + ['[SEP]']  # 이게 특징적이다!!!! bert의 6개 토큰이 들어가면 6개의 임베딩이 출력된다. 
# 하지만 어떤 토큰을 쓴건가 결정? 이 CLS라는 특별한 토크능ㄹ 사용하게 된다. 그래서 실제로는 분절화된 토큰 뿐 아니라 
# 맨 앞의 CLS속성 이용하고 문장이 끝났다는 거르 의마흔 sep을 사용한다seperate의 약자
# 문장이 두개가 입력된다면 무저건 맨 앞ㅇ는 cls하나만 들어가고 txt a, b구분하는 sep가 두개 들어가낟 
# 문장이 하나일땐 scl,1ㄱ,sep1개지만 문장이 두개일땐 sep2는 2개 들어감
print(tokens)

In [0]:
ids = [tokenizer.convert_tokens_to_ids(tokens)]
print(ids) #토크나이저거 토큰을 id로 변환해서 컴퓨터에 숫자로 학습을 시켜야 한다. 

In [0]:
input_tensor = torch.tensor(ids)
print(input_tensor.data)
print(input_tensor.size()) #한문장 8개의 토큰 768개로 표현된다.

##### **Model**

In [0]:
hidden_tensor = model(input_tensor)[0] #구글이 제공하는 텐서. id를 텐서로 변환해주는 과정이 필요하다. 값은 동일한데 (안에)
#  이거를 토치로 같은 텐서라 하더라도 파이토치와 텐서플러우의 텐서는 호환 안된다. 
# 적절한 것으로 변환하는게 필요하다
print(hidden_tensor.size()) 
# 버트의 가장 작은 모델은 768다. 그래서 이거는 지켜줘야 한다.  큰 모델은 1024.. 이거는 뭔가 정해져 있는 값 같다. 

In [0]:
logit = torch.nn.Linear(768, 3)(hidden_tensor) # 2를 3으로 바꾸기도 한다/차원임/
print(logit.size()) # 768을 3으로 클래스가 3개일때 사람이냐 강아지냐 고양이냐. task있을때 이 세개중 어디가 가장 크게 나오느냐를 
# 3으로 변환을 해주는데,,, 0.27ㅇ ㅣ가장 크면 첫번째 클래스를 선택하고, 그렂히 않은 경우 어디 선택하고~~~ 
print(logit.data)

In [0]:
prediction = torch.nn.Softmax(dim=-1)(logit)  # 음수를 0~1사이 확률값으로 조정해주겠다. softmax이용. 합이 1이 됨. 
# 사람이 해석 가능한 값으로 치환한 것 
# forward 방향으로 출력이 진행 됨. 예측된 결과가 얼마나 오차 큰지, 역으로 전파하며 back으로 가는데, 그거 트레이닝 과정 살펴봄
# toturial 어떻게 자연어 변환해서, 최종 클래스를 내느냐를 살펴보았따. 
print(prediction.size())
print(prediction.data)

# **Emotion Recognition**

##### **Dataset**

In [0]:
import json

data = {'train': {'speaker': [], 'utterance': [], 'emotion': []},
        'dev': {'speaker': [], 'utterance': [], 'emotion': []},
        'test': {'speaker': [], 'utterance': [], 'emotion': []}}

for dtype in ['train', 'dev', 'test']:
  for dialog in json.loads(open('friends_' + dtype + '.json').read()):
    for line in dialog:
      data[dtype]['speaker'].append(line['speaker'])
      data[dtype]['utterance'].append(line['utterance'])
      data[dtype]['emotion'].append(line['emotion'])

In [0]:
e2i_dict = dict((emo, i) for i, emo in enumerate(set(data['train']['emotion'])))
i2e_dict = {i: e for e, i in e2i_dict.items()}

##### **Model**

In [0]:
import torch.nn as nn
from transformers import BertModel, BertTokenizer

class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.bert_tokenizer = BertTokenizer.from_pretrained(pretrained_weights) # 자연어를 어떻게 나눌지. 숫자 값으로 변환하고 
    self.bert_model = BertModel.from_pretrained(pretrained_weights)   #버트 모델 768로 나올텐데, 클래스가 우리가 가진 이모션 갯수가 8개
    self.linear = torch.nn.Linear(768, len(e2i_dict)) #버트 모델 중 하나를 예측하게 되는 것 

  def forward(self, utterance):
    tokens = self.bert_tokenizer.tokenize(utterance)
    tokens = ['[CLS]'] + tokens + ['[SEP]'] # (len)
    ids = [tokenizer.convert_tokens_to_ids(tokens)] # (bat=1, len)
    input_tensor = torch.tensor(ids).cuda()

    hidden_tensor = self.bert_model(input_tensor)[0] # (bat, len, hid)
    hidden_tensor = hidden_tensor[:, 0, :] # (bat, hid)
    logit = self.linear(hidden_tensor)
    return logit

##### **Evaluation Metrics**

In [0]:
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate(true_list, pred_list):
  precision = precision_score(true_list, pred_list, average=None)
  recall = recall_score(true_list, pred_list, average=None)
  micro_f1 = f1_score(true_list, pred_list, average='micro')
  print('precision:\t', ['%.4f' % v for v in precision])
  print('recall:\t\t', ['%.4f' % v for v in recall])
  print('micro_f1: %.6f' % micro_f1)
  # 모델이 제대로 측정이 되었냐? 평가 메트릭을 어떻게 설정할 것인가? 얼마나 빠르게 학습 할건지, 몇번을 할건지.. 

##### **Hyper-parameters**

In [0]:
pretrained_weights = 'bert-base-uncased' # base->large로 써도 된다.
# bert-base-uncased ->bert-large-uncased 로 바꿀 경우 위에서 1024로 바꿔야 한다. 
# bert-base-cased 로 써도 성능이 좋아질거다. 
# learning rate 바꿔도 좋아질거고, 에폭도 50, 100 이러게 하면 성능이 높아질거다. 이게 colab이라서 너무 오래 돌면.. 12시간 돌면 초기화 될꺼다. 
learning_rate = 1e-5  # 얼마나 학습을 빠르게 할꺼냐. 너무 빠르면, 성급한 일반화의 오류에,, 세세한 것에는 관심 못가질수도... 
# 적정수준 필요. 작게 잡는게 학습데이이터 많이 ㅣㄹ요하고 느리지만, 오랜시간 하습시 더 정확하다.
n_epoch = 3  # 한 모델이 처음~끝, 한 에폭... 이 데이터를 여러번 읽음. 

##### **Training**

In [0]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import torch
from tqdm import tqdm_notebook

model = Model()
model.cuda() #mbdia GPU 생성하는데, 거기서 Mbida와 이게 호환 되도록 프로그램을 이용해 연결함. 이거를 cuda를 이용해 cpu쓰겠따는 뜻 
criterion = torch.nn.CrossEntropyLoss() # LogSoftmax & NLLLoss  모델의 정확도. 맞췄냐 못맞췄냐..로스로 설정하겠다. 모델에 전달해서 모델 학습한다. 
optimizer = torch.optim.Adam(model.parameters(), learning_rate)

for i_epoch in range(n_epoch):
  print('i_epoch:', i_epoch)

  model.train()
  for i_batch in tqdm_notebook(range(len(data['train']['utterance']))):
    logit = model(data['train']['utterance'][i_batch])
    target = torch.tensor([e2i_dict[data['train']['emotion'][i_batch]]]).cuda()
    loss = criterion(logit, target)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
  

  #  이거를 한 칸씩 들여 넣고, 몇번 했을때마다 모델 평가하는 형태로 바꾸면 조금 더 빠르게 할 수 이다. 
  model.eval()
  pred_list, true_list = [], []
  for i_batch in tqdm_notebook(range(len(data['dev']['utterance']))): # progress bar 이거는 노트북이라서 nb다. 그냥 tqdm notebook으로 하면 잘 나온다. 
    logit = model(data['dev']['utterance'][i_batch])
    _, max_idx = torch.max(logit, dim=-1)
    pred_list += max_idx.tolist()
    true_list += [e2i_dict[data['dev']['emotion'][i_batch]]]
  evaluate(pred_list, true_list) # print results

KeyboardInterrupt: ignored

##### **Labeling**


In [0]:
from collections import OrderedDict

labeled = []
dialogs = json.loads(open('unlabeled.json').read())
for dialog in tqdm_notebook(dialogs):
  dialog_list = []
  for line in dialog:
    logit = model(line['utterance'])
    _, max_idx = torch.max(logit, dim=-1)
    pred_emotion = max_idx.tolist()[0]

    line_dict = OrderedDict()
    line_dict['speaker'] = line['speaker']
    line_dict['utterance'] = line['utterance']
    line_dict['emotion'] = i2e_dict[pred_emotion]
    dialog_list.append(line_dict)
  labeled.append(dialog_list)

with open('labeled.json', 'w') as json_file:
  json.dump(labeled, json_file, indent='\t')

  # 학습하고,, unlabled된거를 레이블링 하는 작업이다. 

##### **Proposal**

- There is a class imbalance problem. (Use weighted cross-entropy etc.)

- Our model takes a single sentence. (Make it grasp its context as well.)

- Our model does not consider speaker information. (Make it consider the info.)

- Batch size is set as 1. (Increase the batch size.)

In [0]:
# 학습해서 labled를 내고, 파이썬 파일 수정한거 내고, 레포트는 각자 조사한거 제출하면 된다. 

# technical report :  버트 내부가 어덯게 하기때문에 이걸 사용했따던지.. 구체적으로 적어줄수록..좋을 것 같다. 
# 자료의 구성과 흐름... 성의... 기승전결.. 얼마나 잘 이뤄졌느냐. 이문제가 머가 왜 이걸 썻고, 그래서 어떻게 해결했꼬, 어떤 문제가 있다는 형태의 
# 논문형태로 써주면 좋을 것 같습니다. 

# proposal 은 참고하라고 적어 주신거고. 알아서 개인적으로 의견 넣으면 된다. 

# 감정이 굉장히 biased 되어 있는데, 이거를 어떻게 균형잡히게 하면 될지, 
# crossendtrophyloss(weight....) 이렇게 넣으면 더 늘어난다. 
# 문장이 아니라 문맥을
# 스피처.. 배치 사이즈를 늘려서 여러개를 종합해서,, 
# 로스를 개선학ㅆ다는 거라서 그거를 키우면 학습이 잘될꺼다. 개선점들을 바꿔봐라 
# 화자가 누구인지를 넣으면 성능이 더 좋아질 수 있다. 

# jay alammar 의 딥러닝... 이거 참고할 것